# Loader

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import make_scorer
import xgboost as xgb
from scipy.optimize import curve_fit
from scipy.fft import fft, fftfreq
from scipy.stats import mode
import tensorflow as tf
import irp_utils
import time

2024-08-01 19:48:47.730248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 19:48:47.730395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 19:48:47.940950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!cat /proc/cpuinfo 

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 6

In [3]:
## Install wandb
!pip install wandb

# Import necessary libraries
import wandb
from kaggle_secrets import UserSecretsClient



secrets = UserSecretsClient()


In [4]:
wandb_user = secrets.get_secret("wandb_user")
wandb_pass = secrets.get_secret("wandb-secret-key")
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

KeyboardInterrupt



In [ ]:
wandb.init(entity=wandb_user)
init_time = time.time()

In [ ]:
time_table = wandb.Table(columns=["Event","Time Since Initialization (seconds)"])

def log_time(event_name):
    elapsed_time = time.time() - init_time
    time_table.add_data(event_name,elapsed_time)
    wandb.log({event_name: elapsed_time})

## Choose Mission

In [ ]:
mission_choice = 2

channels_to_load = [75,76,77,83,84,85]

#1

#[12,13,14] #1 Group 3, Doesnt Work - good score but different resolutions for anomalous sections
#[12,13,19,20,27,28,36,37] # 1 Group 3, Doesnt Work
#[27,28,36,37] 1 Group 3, works
#[14,21,29] #1 Group 4, Works
#[14,21,29,30,38] # 1 Group 4, Works
#[17,18,25,26,34,35] # 1 Group 5, Doesnt Work - good score but different resolutions for anomalous sections
#[16,24,32,33,40] # 1 Group 6, Works
#[15,22,23,31,39] # 1 Group 7, Works
#[41,42,43,44,45,46] # 1 Group 8, Works
#[47,48,49] #1 Group 9, Works : 55
#[50] 1 Group 10, Works
#[50,51,52] # 1 Group 10, Works
#[57,58,59,60] # 1 Group 13, Works (doesnt work with mem)
#[61,62,63] # 1 Group 14 Requires slice with anomaly in to train
#[64,65] # 1 Group 15, Works but requires larger slice of data

#2


#[9,10,11,12,13,14] Doesnt Work
#[9,10,11,12,13] works
#[25,26,27,28] # 2 Group 1, Doesnt work
#[21,22,23,24] 2 Group 2,Works: 520
#[18,19,20] # 2 Group 5,Works 680
#[15,16,17] # 2 Group 8, Works
#[73,81] # 2 Group 11, Need to find anomaly in the data slice
#[75,76,77,83,84,85] # 2 Group 31, Need to find anomaly in the data slice

### Load data

In [ ]:
log_time("Dataset Loading Start")

# Initialize an empty DataFrame to store all channels' data
data = pd.DataFrame()

# Load data for each channel and add to the DataFrame
for channel_choice in channels_to_load:
    # File path
    file_path = f'/kaggle/input/esa-anomaly-dataset/ESA-Mission{mission_choice}/ESA-Mission{mission_choice}/channels/channel_{channel_choice}/channel_{channel_choice}'

    # Open the file and load it with pickle
    with open(file_path, 'rb') as file:  # Note 'rb' mode for reading binary files
        channel_data = pickle.load(file)
    
    # Rename the column to include channel number
    channel_data.rename(columns={f'channel_{channel_choice}': f'channel_{channel_choice}'}, inplace=True)
    
    # Merge with the main data DataFrame
    if data.empty:
        data = channel_data
    else:
        data = data.merge(channel_data, left_index=True, right_index=True, how='outer')

# Display the DataFrame
display(data.head())
print(f"Length of merged data: {len(data)}")

### Load Labels

In [ ]:
# Load the labels data
labels = pd.read_csv(f'/kaggle/input/esa-anomaly-dataset/ESA-Mission{mission_choice}/ESA-Mission{mission_choice}/labels.csv')

# Convert datetime columns correctly using .loc to avoid SettingWithCopyWarning
labels.loc[:, 'Start Time'] = pd.to_datetime(labels['StartTime'])
labels.loc[:, 'End Time'] = pd.to_datetime(labels['EndTime'])

# Convert to datetime and remove timezone information to make them timezone-naive
labels['StartTime'] = pd.to_datetime(labels['StartTime']).dt.tz_localize(None)
labels['EndTime'] = pd.to_datetime(labels['EndTime']).dt.tz_localize(None)

# Initialize the 'labels' column to 0 (no anomaly) if not already done
if 'labels' not in data.columns:
    data['labels'] = 0

# Iterate through each channel and update the labels
for channel_choice in channels_to_load:
    # Filter labels for the specific channel
    anomalies = labels[labels['Channel'] == f'channel_{channel_choice}'].copy()
    
    # Update the labels for anomalies
    for _, row in anomalies.iterrows():
        # Get the boolean mask where the index is within the anomaly span
        mask = (data.index >= row['StartTime']) & (data.index <= row['EndTime'])
        
        # Set the label to 1 for anomalies (without overwriting existing 1's)
        data.loc[mask, 'labels'] = data.loc[mask, 'labels'].apply(lambda x: 1 if x == 0 else x)

# Display the head of the dataset to verify labels
display(data.head())

### Plot Dataset

In [ ]:
# # Plot the data and anomalies for each channel
# plt.figure(figsize=(15, 7))
# for channel_choice in channels_to_load:
#     plt.plot(data.index, data[f'channel_{channel_choice}'], label=f'Channel {channel_choice}', linestyle='-')

# # Plot anomalies
# for _, row in anomalies.iterrows():
#     plt.axvspan(row['Start Time'], row['End Time'], color='red', alpha=0.3, label='Anomaly')

# # Handle legend labels (to avoid duplicate labels in the legend)
# handles, labels = plt.gca().get_legend_handles_labels()
# by_label = dict(zip(labels, handles))
# plt.legend(by_label.values(), by_label.keys())

# plt.title('Measurements Over Time with Anomalies for Multiple Channels')
# plt.xlabel('Datetime')
# plt.ylabel('Measurement Value')
# plt.grid(True)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [ ]:
full_machine_data = data[[f'channel_{channel_choice}' for channel_choice in channels_to_load]].copy()  # Only data
full_machine_labels = data[['labels']].copy()    # Only labels
# Verify that both DataFrames are correctly set up
print(full_machine_data.head())
print(full_machine_labels.head())

full_machine_data = full_machine_data.fillna(0)
#full_machine_labels = full_machine_data.fillna(1)

log_time("Dataset Loading End")

## Sectioning Data

In [ ]:
log_time("Sectioning Dataset Start")
# Calculate the index for the 20% slice
slice_size = int(len(full_machine_data) * 0.08)


# Take a contiguous slice of the data and labels
machine_data = full_machine_data.iloc[:slice_size]
machine_labels = full_machine_labels.iloc[:slice_size]
                                            
print("Training Data Shape:", machine_data.shape)

log_time("Sectioning Dataset End")


In [ ]:
# Plot the data and anomalies for each channel
plt.figure(figsize=(15, 7))

# Plot anomalies
anomaly_indices = machine_labels[machine_labels['labels'] == 1].index
for anomaly_index in anomaly_indices:
    plt.axvline(x=anomaly_index, color='red', alpha=0.1, label='Anomaly')

for channel_choice in channels_to_load:
    plt.plot(machine_data.index, machine_data[f'channel_{channel_choice}'], label=f'Channel {channel_choice}', linestyle='-')
    
# Handle legend labels (to avoid duplicate labels in the legend)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.title('Measurements Over Time with Anomalies for Multiple Channels')
plt.xlabel('Datetime')
plt.ylabel('Measurement Value')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Get Sub-Cycles

In [ ]:
sub_cycle_divisor = 6
max_cycle_length = 500
min_cycle_length = 100

user_defined_cycle_length = False
use_average_cycle_length = False
use_most_common_cycle_length = True
use_individual_cycle_length = False

In [ ]:
log_time("Getting Sub-Cycles Start")

# Initialize the dictionary to store sub-cycle outputs
sub_cycles = {}
cycle_lengths = []

irp_utils.validate_cycle_length_selection(user_defined_cycle_length, use_average_cycle_length, use_most_common_cycle_length, use_individual_cycle_length)

if not user_defined_cycle_length:
    # Loop through all features and store the sub-cycle outputs
    for feature in range(machine_data.shape[1]):
        nominal_sub_cycles, anomalous_sub_cycles, cycle_length = irp_utils.get_subcycle(feature, sub_cycle_divisor, machine_data, 
                                                                              machine_labels, max_cycle_length=max_cycle_length,
                                                                              min_cycle_length=min_cycle_length,plot_results=False)

        if cycle_length is not None:
            cycle_lengths.append(cycle_length)

        # Initialize the sub-cycles for the current feature
        sub_cycles[feature] = {
            'nominal': [],
            'anomalous': []
        }

        if not user_defined_cycle_length and not use_average_cycle_length and not use_most_common_cycle_length:
            # Append the nominal and anomalous sub-cycles for the current feature
            sub_cycles[feature]['nominal'].append(nominal_sub_cycles)
            sub_cycles[feature]['anomalous'].append(anomalous_sub_cycles)

        

# Filter out None values from cycle_lengths
cycle_lengths = [cl for cl in cycle_lengths if cl is not None]

if user_defined_cycle_length or use_average_cycle_length or use_most_common_cycle_length:


    if user_defined_cycle_length:
        final_cycle_length = user_defined_cycle_length
        
    elif use_average_cycle_length:
        final_cycle_length = int(np.mean(cycle_lengths))
        
    elif use_most_common_cycle_length:
        counts = np.bincount(cycle_lengths)
        final_cycle_length = np.argmax(counts)
        print(f"FINAL:{final_cycle_length}")
        
    elif use_individual_cycle_length:
        final_cycle_length = None
    else:
        final_cycle_length = max_cycle_length

    print(f"Final Cycle Length: {final_cycle_length}")
    
    # Apply the final cycle length to all features
    for feature in tqdm(range(machine_data.shape[1])):
        
        # Initialize the sub-cycles for the current feature
        sub_cycles[feature] = {
            'nominal': [],
            'anomalous': []
        }
        
        nominal_sub_cycles, anomalous_sub_cycles, _ = irp_utils.get_subcycle(feature, sub_cycle_divisor, machine_data, 
                                                                   machine_labels, max_cycle_length=max_cycle_length, 
                                                                   min_cycle_length=min_cycle_length,
                                                                   plot_results=False, user_cycle_length=final_cycle_length,
                                                                   use_individual_cycle_lengths=use_individual_cycle_length)

        sub_cycles[feature]['nominal'].append(nominal_sub_cycles)
        sub_cycles[feature]['anomalous'].append(anomalous_sub_cycles)
        
log_time("Getting Sub-Cycles End")


In [ ]:
print(f"Cycle Lengths: {cycle_lengths}")

sub_cycle_lengths = [int(x / sub_cycle_divisor) for x in cycle_lengths]
print(f"Sub-Cycle Lengths: {sub_cycle_lengths}")

In [ ]:
# Assuming sub_cycles is a dictionary with the required data
num_features = len(sub_cycles)

for feature in range(num_features):
    fig, axes = plt.subplots(2, sub_cycle_divisor, figsize=(18, 12), sharex=True)
    
    for i in range(sub_cycle_divisor):  # There are sub-cycles for nominal and anomalous
        # Plot nominal sub-cycles for the current feature
        for cycle in sub_cycles[feature]['nominal'][0][i]:
            if isinstance(cycle, (list, np.ndarray)):
                axes[0, i].plot(np.arange(len(cycle)), cycle)
        axes[0, i].set_title(f'Nominal SC {i+1} for Ch {feature}')
        axes[0, i].set_ylabel('Normalized Value')
        axes[0, i].axhline(y=0, color='black', linestyle='--', linewidth=1)
        
        # Plot anomalous sub-cycles for the current feature
        for cycle in sub_cycles[feature]['anomalous'][0][i]:
            if isinstance(cycle, (list, np.ndarray)):
                axes[1, i].plot(np.arange(len(cycle)), cycle)
        axes[1, i].set_title(f'Anomalous SC {i+1} for Ch {feature}')
        axes[1, i].set_ylabel('Normalized Value')
        axes[1, i].axhline(y=0, color='black', linestyle='--', linewidth=1)
    
    for ax in axes[1, :]:
        ax.set_xlabel('Time')
    
    plt.tight_layout()
    plt.show()

In [ ]:
log_time("Getting Features Start")

# Calculate similarities
nom_comparisons = 90
anom_comparisons = 30
nominal_similarities, anomalous_similarities = irp_utils.calculate_similarity(sub_cycles,nom_comparisons,anom_comparisons,sub_cycle_divisor)

# Convert to DataFrames
anomalous_df = pd.DataFrame(anomalous_similarities)
nominal_df = pd.DataFrame(nominal_similarities)

log_time("Getting Features End")


In [ ]:
# Display DataFrames
print("Anomalous Sub-cycle Similarities:")
anomalous_df = anomalous_df.fillna(0)
anomalous_df

In [ ]:
print("Nominal Sub-cycle Similarities:")
nominal_df = nominal_df.fillna(0)
nominal_df

In [ ]:
anomalous_data_loaded = anomalous_df.drop(columns=['comparison_type'])#,'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])
nominal_data_loaded = nominal_df.drop(columns=['comparison_type'])#,'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])

# anomalous_data_loaded = anomalous_data_loaded.drop(columns=['euclidean_distance','rmse', 'std_diff', 'min_diff', 'max_diff', 'mean_diff', 'fft_diff','var_diff',
#             'mad_diff','iqr_diff','mad_mean_diff','energy_diff','skew_diff','kurtosis_diff','spectral_entropy_diff'])#'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])
# nominal_data_loaded = nominal_data_loaded.drop(columns=['euclidean_distance','rmse', 'std_diff', 'min_diff', 'max_diff', 'mean_diff', 'fft_diff','var_diff',
#             'mad_diff','iqr_diff','mad_mean_diff','energy_diff','skew_diff','kurtosis_diff','spectral_entropy_diff'])#'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])

# Training

In [ ]:
# Prediction imports
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout,BatchNormalization,Lambda,InputLayer, GaussianNoise, add
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tqdm.notebook import tqdm
import re

from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,f1_score, make_scorer,precision_recall_fscore_support
from sklearn.impute import SimpleImputer
import time
import random
import gc
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
nominal_df = nominal_data_loaded
anomalous_df = anomalous_data_loaded

In [ ]:
# Ensure TensorFlow uses the GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
gc.collect()

In [ ]:
# List of features for training
features = ['pearson_correlation', 'spearman_correlation', 'euclidean_distance', 'cosine_similarity', 
            'rmse', 'std_diff', 'min_diff', 'max_diff', 'mean_diff', 'fft_diff','var_diff',
            'mad_diff','iqr_diff','mad_mean_diff','energy_diff','skew_diff','kurtosis_diff','spectral_entropy_diff','rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff']#,'dtw_distance']

In [ ]:
# Separate the data by channels and split before any processing
unique_channels = nominal_df['feature'].unique()
split_data = {}

# Split the data for each channel
print("Splitting data by channels")
for channel in tqdm(unique_channels):
    #print(f"Channel {channel}")
    
    # Filter the data for the current feature
    nominal_data = nominal_df[nominal_df['feature'] == channel]
    anomalous_data = anomalous_df[anomalous_df['feature'] == channel]

    # Drop unnecessary columns
    #nominal_data = nominal_data.drop(columns=['nominal_cycle_1', 'nominal_cycle_2'])#,'target']) #, 'feature'])
    #anomalous_data = anomalous_data.drop(columns=['target'])
    
    # Split the nominal data into training, validation, and test sets
    X_train, X_val, X_test_nominal = irp_utils.split_nominal_data(nominal_data)

    # Store the split data
    split_data[channel] = {
        'X_train': X_train,
        'X_val': X_val,
        'X_test_nominal': X_test_nominal,
        #'X_threshold': X_threshold,
        'anomalous_data': anomalous_data
    }

In [ ]:

log_time("Training Start")

# Train autoencoders and prepare test data
autoencoders = []
val_data_list = []
X_test_combined = []
histories = []
scalers = []

autoencoders_dict = {}

print("Creating Autoencoders")

# Record start time
start_time = time.time()

for channel in tqdm(unique_channels):

    #print(f"Channel {channel}")
    
    data = split_data[channel]
    
    # Normalize the data
    #scaler = StandardScaler()
    scaler = MinMaxScaler()
    
    X_train_scaled = scaler.fit_transform(data['X_train'])
    X_val_scaled = scaler.transform(data['X_val'])
    X_test_nominal_scaled = scaler.transform(data['X_test_nominal'])
    anomalous_data_scaled = scaler.transform(data['anomalous_data'])
    
    #Put scaler in the list
    scalers.append(scaler)
    
    # Train autoencoder for the feature

    autoencoder, X_val, history = irp_utils.train_autoencoder(X_train_scaled,X_val_scaled,autoencoders_dict, channel)

    autoencoders.append(autoencoder)
    val_data_list.append(X_val_scaled)  # Use X_val_scaled to ensure consistency
    #threshold_data_list.append(X_threshold_scaled)
    histories.append(history)
    
    # Prepare test data for the feature
    X_test = np.concatenate([X_test_nominal_scaled, anomalous_data_scaled])
    X_test_combined.append(X_test)

# Record end time and calculate duration
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration}")

# Combine the test labels for nominal and anomalous sets
y_test = np.concatenate([np.zeros(data['X_test_nominal'].shape[0]), np.ones(data['anomalous_data'].shape[0])])

# Shuffle the test set
test_indices = np.arange(len(y_test))
np.random.shuffle(test_indices)
X_test_combined = [X_test[test_indices] for X_test in X_test_combined]
y_test = y_test[test_indices]

log_time("Training End")

# Evaluation

In [ ]:
log_time("Evaluation Start")

print("Evaluating Autoencoders")

# Create a figure for the subplots
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(30, 15))  # Adjust the size as necessary
axes = axes.flatten()

# Loop through thresholds from 50 to 100
for i, percentile_threshold in enumerate(range(50, 101,10)):
    # Record start time
    start_time = time.time()
    
    #results_df, f1c, f1, precision, recall, f1pa, threshold
    # Evaluate the ensemble of autoencoders
    results_df, f1c, f1, precision, recall, corrected_f1,channel_f1,alarming_prec,adtqc_score,affiliation_f1, threshold= irp_utils.evaluate_ensemble(
        autoencoders, X_test_combined, y_test, val_data_list, percentile_threshold
    )

    # Generate the confusion matrix
    cm = confusion_matrix(results_df['Actual Label'], results_df['Predicted Anomaly'])
    
    # Plot the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Nominal', 'Anomalous'])
    disp.plot(ax=axes[i], colorbar=False)
    axes[i].set_title(f"Threshold: {percentile_threshold}")

    # Record end time and calculate duration
    end_time = time.time()
    evaluation_duration = end_time - start_time

    # Print results (optional)
    print(f"Threshold: {percentile_threshold}")
    print(f"Evaluation time: {evaluation_duration}")
    print(f"F1C: {f1c}, F1: {f1}, Precision: {precision}, Recall: {recall}, Threshold used: {percentile_threshold}")
    print("="*50)

    
log_time("Evaluation End")
# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
print("Evaluating Autoencoders")
# Record start time
start_time = time.time()
percentile_threshold = 100
# Evaluate the ensemble of autoencoders
results_df, f1c, f1, precision, recall, corrected_f1,channel_f1,alarming_prec,adtqc_score,affiliation_f1, threshold = irp_utils.evaluate_ensemble(autoencoders, X_test_combined, 
                                                                                      y_test, val_data_list,
                                                                                      percentile_threshold)
# Record end time and calculate duration
end_time = time.time()
evaluation_duration = end_time - start_time
print(f"Evaluation time: {evaluation_duration}")

In [ ]:
# Plot the results and confusion matrix
plt.figure(figsize=(14, 7))

# Plot reconstruction error
plt.subplot(4, 1, 1)
plt.plot(results_df['Reconstruction Error'], label='Reconstruction Error')
plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
plt.title(f'Reconstruction Error and Threshold')
plt.legend()

# # Plot actual vs predicted
# plt.subplot(4, 1, 2)
# plt.plot(results_df['Actual Label'], label='Actual Label', alpha=0.7)
# plt.plot(results_df['Predicted Anomaly'], label='Predicted Anomaly', alpha=0.3)
# plt.title(f'Actual Labels vs Predicted Anomalies')
# plt.legend()

# Plot actual vs predicted highlighting correct and incorrect predictions
plt.subplot(4, 1, 2)
correct_predictions = results_df[results_df['Correct Prediction'] == True]
incorrect_predictions = results_df[results_df['Correct Prediction'] == False]

#plt.plot(correct_predictions.index, correct_predictions['Actual Label'], 'go', markersize=4, label='Correct Predictions')
#plt.plot(incorrect_predictions.index, incorrect_predictions['Actual Label'], 'ro', markersize=4, label='Incorrect Predictions')
offset = 0.2
plt.plot(correct_predictions.index, correct_predictions['Actual Label'], 'go', markersize=4, label='Correct Predictions')
plt.plot(incorrect_predictions.index, incorrect_predictions['Actual Label'] + offset, 'ro', markersize=4, label='Incorrect Predictions')

plt.title(f'Actual Labels vs Predicted Anomalies (Correct and Incorrect Predictions)')
plt.legend()

# Plot confusion matrix
plt.subplot(4, 1, 3)
cm = confusion_matrix(results_df['Actual Label'], results_df['Predicted Anomaly'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Nominal', 'Anomalous'])
disp.plot(ax=plt.gca())
plt.title(f'Confusion Matrix')


plt.tight_layout()
plt.show()

print(f'f1c Score: {f1c}')
print(f'f1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
#print(f"f1pa Score: {f1pa}")
print(f'f1_corrected: {corrected_f1}')
print(f'Channel Aware f1: {channel_f1}')
print(f'Event Wise Alarming Precision: {alarming_prec}')
print(f'ADTQC: {adtqc_score}')
#print(f'Affiliation-Based f1: {affiliation_f1}')

# Plot the training stats
plt.figure(figsize=(14, 7))
for i, history in enumerate(histories):
    plt.plot(history.history['loss'], label=f'Channel {unique_channels[i]} - Training Loss')
plt.title('Training Loss for All Channels')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4)
plt.show()

# Plot the training stats
plt.figure(figsize=(14, 7))
for i, history in enumerate(histories):
    plt.plot(history.history['val_loss'], label=f'Channel {unique_channels[i]} - Validation Loss')
plt.title('Validation Loss for All Channels')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4)
plt.show()

results_df

In [ ]:
autoencoders[0].summary()

In [ ]:
from tensorflow.keras.utils import plot_model

# Plot a model and save to a file
plot_model(autoencoders[0], to_file='autoencoder_model_plot.png', show_shapes=True, show_layer_names=True)

# Display the image
from IPython.display import Image
Image('autoencoder_model_plot.png')

# Testing on Live Data

In [ ]:

def get_subcycle_with_length(feature, sub_cycle_length, machine_data, 
                             max_freq=1.0, initial_step=0.01, refinement_steps=3, 
                             max_cycle_length=1500, min_cycle_length=400, plot_results=True):
    """
    Function to sort subcycles into arrays using a provided sub-cycle length.
    Input: Chosen feature channel (int), Sub-cycle length (int), Data.
    Output: Subcycles.
    """
    print(f"Feature {feature}")

    def estimate_cycle_length(y_data, step, already_checked_freqs):
        offset_estimate = y_data.mean()
        yf = fft(y_data - offset_estimate)
        xf = fftfreq(len(y_data), 1)
        half_len = len(yf) // 2
        yf = yf[:half_len]
        xf = xf[:half_len]
        valid_range = xf <= max_freq
        yf = yf[valid_range]
        xf = xf[valid_range]
        mask = np.isin(xf, already_checked_freqs, invert=True)
        yf = yf[mask]
        xf = xf[mask]
        if len(xf) == 0:
            return None, None
        coarse_index = np.arange(0, len(xf), max(1, int(step)))
        dominant_freq_index = coarse_index[np.argmax(2.0 / len(y_data) * np.abs(yf[coarse_index]))]
        return xf[dominant_freq_index], dominant_freq_index

    def find_cycle_length(y_data):
        already_checked_freqs = []
        dominant_freq, _ = estimate_cycle_length(y_data, initial_step, already_checked_freqs)
        if dominant_freq:
            already_checked_freqs.append(dominant_freq)
        
        for _ in range(refinement_steps):
            initial_step = max(1, initial_step // 2)
            new_freq, _ = estimate_cycle_length(y_data, initial_step, already_checked_freqs)
            if new_freq:
                dominant_freq = new_freq
                already_checked_freqs.append(dominant_freq)

        cycle_length = int(1 / dominant_freq) if dominant_freq else max_cycle_length

        while (cycle_length < min_cycle_length or cycle_length > max_cycle_length) and dominant_freq:
            if cycle_length < min_cycle_length:
                new_freq, _ = estimate_cycle_length(y_data, initial_step, already_checked_freqs)
                if new_freq:
                    dominant_freq = new_freq
                    cycle_length = int(1 / dominant_freq)
                    already_checked_freqs.append(dominant_freq)
            elif cycle_length > max_cycle_length:
                new_freq, _ = estimate_cycle_length(y_data, initial_step, already_checked_freqs)
                if new_freq:
                    dominant_freq = new_freq
                    cycle_length = int(1 / dominant_freq)
                    already_checked_freqs.append(dominant_freq)
            else:
                break
        
        return cycle_length

    x_data = np.arange(len(machine_data))
    y_data = machine_data.iloc[:, feature].to_numpy()

    # Use the provided sub-cycle length directly
    cycle_length = sub_cycle_length

    print(f"Cycle Length: {cycle_length}")

    if plot_results and cycle_length >= min_cycle_length and cycle_length <= max_cycle_length:
        plt.figure(figsize=(10, 6))
        plt.plot(x_data, y_data, label='Data')
        already_checked_freqs = []
        dominant_freq, _ = estimate_cycle_length(y_data, initial_step, already_checked_freqs)
        if dominant_freq:
            already_checked_freqs.append(dominant_freq)
            initial_guess = [(y_data.max() - y_data.min()) / 2, 2 * np.pi * dominant_freq, 0, y_data.mean()]
            try:
                params, _ = curve_fit(sine_wave, x_data, y_data, p0=initial_guess, maxfev=2000)
                plt.plot(x_data, sine_wave(x_data, *params), color='lime', label='Fitted Sine Wave')
            except RuntimeError as e:
                print(f"Curve fitting failed: {e}")
        for i in np.arange(0, len(x_data), cycle_length):
            plt.axvline(x=i, color='black', linestyle='-', linewidth=1.5)
        for i in np.arange(0, len(x_data), cycle_length):
            plt.axvline(x=i, color='black', linestyle='--', linewidth=1)
        plt.title(f'Sine Wave Fitting to Feature {feature}')
        plt.xlabel('Time')
        plt.ylabel('Normalized Value')
        plt.legend()
        plt.show()

    sub_cycles = []

    for start in range(0, len(x_data), cycle_length):
        sub_cycle_end = start + cycle_length
        if sub_cycle_end <= len(x_data):
            sub_cycle_data = y_data[start:sub_cycle_end]
            sub_cycles.append((start,sub_cycle_data))

    return sub_cycles, cycle_length

# Define the sine wave function used for curve fitting
def sine_wave(x, A, B, C, D):
    return A * np.sin(B * x + C) + D

In [ ]:
start_index = 100
final_sub_cycle_length = int(final_cycle_length/sub_cycle_divisor)

start_slice = slice_size + final_sub_cycle_length*start_index
end_slice = start_slice + final_sub_cycle_length*70
# Take a new slice which will be used to simulate live data
live_data = full_machine_data.iloc[start_slice:end_slice]
live_labels = full_machine_labels.iloc[start_slice:end_slice]

print("Live Data Shape:", live_data.shape)
print(live_data)

In [ ]:
# Plot the data and anomalies for each channel
plt.figure(figsize=(15, 7))

sub_cycle_plot_length = final_sub_cycle_length
# Add vertical blue lines every final_sub_cycle_length
for j in range(0, len(live_data),final_sub_cycle_length):
    plt.axvline(x=live_data.index[j], color='blue', linestyle='--', alpha=0.7, label='Sub Cycle')


# Plot anomalies
anomaly_indices = live_labels[live_labels['labels'] == 1].index
for anomaly_index in anomaly_indices:
    plt.axvline(x=anomaly_index, color='red', alpha=0.1, label='Anomaly')

for channel_choice in channels_to_load:
    plt.plot(live_data.index, live_data[f'channel_{channel_choice}'], label=f'Channel {channel_choice}', linestyle='-')
    
# Handle legend labels (to avoid duplicate labels in the legend)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.title('Measurements Over Time with Anomalies for Multiple Channels')
plt.xlabel('Datetime')
plt.ylabel('Measurement Value')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def calculate_similarity(sub_cycles, live_sub_cycle, feature, sub_cycle_index, max_comparisons):
    similarities = []

    #print("Calculating Similarities")

    comparison_count = 0
    #nominal_cycles = sub_cycles[feature]['nominal'][0][sub_cycle_index]
    
    nominal_cycles = sub_cycles[feature]['nominal'][0][sub_cycle_index][::-1]  # Reverse the nominal cycles

    #print(f"Channel: {feature}, Sub Cycle: {sub_cycle_index}\n Nominal Cycles:{nominal_cycles}, Live Cycles: {live_sub_cycle}")

    total_comparisons = len(nominal_cycles) * max_comparisons

    #with tqdm(total=total_comparisons, desc="Comparisons") as pbar:
    # Compare the live sub-cycle with nominal cycles
    for nom_cycle in nominal_cycles:
        if comparison_count >= max_comparisons:
            break
        if len(live_sub_cycle) == len(nom_cycle):
            similarity = irp_utils.compute_similarity(live_sub_cycle, nom_cycle, feature, sub_cycle_index, "live vs nominal")
            similarities.append(similarity)
            comparison_count += 1
            #pbar.update(1)
    #print(comparison_count)
    return similarities


start_slice = slice_size+(final_sub_cycle_length)*start_index

max_comparisons = 50#200
sub_cycle_index=0
split_data = {}

# DataFrame for accumulating data over time
accumulated_df = pd.DataFrame()

# Initialize the plot
fig, ax = plt.subplots(figsize=(20, 10))


log_time("Live Test Start")


# Loop to simulate the live update
for i in range(1000):  # Adjust the range for the number of updates you want
    start_time = time.time()
    print(start_slice)
    # Move slice along
    start_slice += final_sub_cycle_length# * 6              #if *6, this is 6 sub_cycles
    end_slice = start_slice + final_sub_cycle_length# * 6   #if *6, this is 6 sub_cycles
    
    live_data = full_machine_data.iloc[start_slice:end_slice]
    live_labels = full_machine_labels.iloc[start_slice:end_slice]
    
    j = 0

    # Combine reconstruction errors from all autoencoders
    reconstruction_errors = []
    channel_errors = {channel: [] for channel in channels_to_load}  # Dictionary to store errors for each channel

    # DO THING WITH LIVE DATA SLICE HERE
    for channel_choice in channels_to_load:
        # For each channel

        print(f"Comparing channel {channel_choice}, column index {j}")
        channel_data = live_data[f'channel_{channel_choice}'].values
        #print(channel_data.shape)

        if end_slice - start_slice > final_sub_cycle_length:
            # Split the channel data into sub-cycles
            sub_cycles_data = np.array_split(channel_data, sub_cycle_divisor)
            for sub_cycle_index in range(sub_cycle_divisor):
                
                sub_cycle_data = sub_cycles_data[sub_cycle_index]
                
                similarities = calculate_similarity(sub_cycles, sub_cycle_data, j, sub_cycle_index, max_comparisons)
                # Convert similarities to DataFrame
                similarities_df = pd.DataFrame(similarities).fillna(0)
                #similarities_df = similarities_df.fillna(0)
                similarities_data = similarities_df.drop(columns=['comparison_type'])#,'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])
                #similatities_data = similarities_data.drop(columns=['euclidean_distance','rmse', 'std_diff', 'min_diff', 'max_diff', 'mean_diff', 'fft_diff','var_diff',
                #'mad_diff','iqr_diff','mad_mean_diff','energy_diff','skew_diff','kurtosis_diff','spectral_entropy_diff'])
                #similarities_data = similarities_df
                # Scale the data for autoencoder
                data_for_autoencoder = scalers[j].transform(similarities_data)
                # Autoencoder predictions and reconstruction error
                test_predictions = autoencoders[j].predict(data_for_autoencoder)
                reconstruction_error = np.mean(np.square(data_for_autoencoder - test_predictions), axis=1)
                reconstruction_errors.append(reconstruction_error)
                channel_errors[channel_choice].append(np.mean(reconstruction_error))


    
        else:
            similarities = calculate_similarity(sub_cycles, channel_data, j, sub_cycle_index, max_comparisons)
            # Convert similarities to DataFrame
            similarities_df = pd.DataFrame(similarities).fillna(0)
            #similarities_df = similarities_df.fillna(0)
            similarities_data = similarities_df.drop(columns=['comparison_type'])#,'rolling_mean_diff','rolling_std_diff','fft_mean_diff','fft_std_diff','fft_max_diff'])
            #similatities_data = similarities_data.drop(columns=['euclidean_distance','rmse', 'std_diff', 'min_diff', 'max_diff', 'mean_diff', 'fft_diff','var_diff',
            #'mad_diff','iqr_diff','mad_mean_diff','energy_diff','skew_diff','kurtosis_diff','spectral_entropy_diff'])
            #similarities_data = similarities_df
            # Scale the data for autoencoder
            data_for_autoencoder = scalers[j].transform(similarities_data)
            # Autoencoder predictions and reconstruction error
            test_predictions = autoencoders[j].predict(data_for_autoencoder)
            reconstruction_error = np.mean(np.square(data_for_autoencoder - test_predictions), axis=1)
            reconstruction_errors.append(reconstruction_error)
            channel_errors[channel_choice].append(np.mean(reconstruction_error))

            # Check if the anomaly percentage is under 50%
            anomaly_percentage = np.mean(reconstruction_errors[-1] > threshold) * 100
            if anomaly_percentage < 50:
                print("Appending nominal data")
                sub_cycles[j]['nominal'].append(channel_data)


            sub_cycle_index += 1
            if sub_cycle_index >= sub_cycle_divisor:
                sub_cycle_index = 0

        j += 1
    # Truncate reconstruction errors to the length of the shortest array
    min_length = min(len(err) for err in reconstruction_errors)
    truncated_reconstruction_errors = [err[:min_length] for err in reconstruction_errors]

    # Average reconstruction errors
    avg_reconstruction_error = np.mean(np.vstack(truncated_reconstruction_errors), axis=0)

    # Identify anomalies
    anomalies = avg_reconstruction_error > threshold
    print(anomalies)
    
        
#     # Reset sub_cycle_index after processing all channels
#     if sub_cycle_index >= 6:
#         sub_cycle_index = 0
        
    
    # Calculate the percentage of anomalies
    anomaly_percentage = np.mean(anomalies) * 100
    print(f"Anomaly percentage: {anomaly_percentage:.2f}%")
    #anomaly_percentages.append(anomaly_percentage)
    
    # Append current data to accumulated DataFrame
    live_data = live_data.copy()
    live_data['labels'] = live_labels['labels']
    live_data['anomaly_percentage'] = anomaly_percentage
    
    # Add channel error ratios and percentages to DataFrame
    total_error = sum(channel_errors[channel_choice][-1] for channel_choice in channels_to_load)
    for channel_choice in channels_to_load:
        if total_error > 0:
            live_data[f'error_ratio_{channel_choice}'] = channel_errors[channel_choice][-1] / total_error
            
        else:
            live_data[f'error_ratio_{channel_choice}'] = 0
            
        live_data[f'error_{channel_choice}'] = channel_errors[channel_choice][-1] if channel_errors[channel_choice] else np.nan



    accumulated_df = pd.concat([accumulated_df, live_data])
    
        # Keep only the latest 12 loops of data
    if len(accumulated_df) > final_sub_cycle_length * 12:
        accumulated_df = accumulated_df.iloc[-final_sub_cycle_length * 12:]
        
    # Record end time and calculate duration
    end_time = time.time()
    evaluation_duration = end_time - start_time
    print(f"Evaluation time: {evaluation_duration}")
    
    if i == 1:
        log_time("Live Test Prediction")
        # Log the table to wandb
        wandb.log({"Time Markers": time_table})


###################################################
    # Clear and update the plot
    ax.clear()
    
    # Plot anomalies
    anomaly_indices = accumulated_df[accumulated_df['labels'] == 1].index
    for anomaly_index in anomaly_indices:
        ax.axvline(x=anomaly_index, color='red', alpha=0.3, label="Ground Truth Anomaly")

    # Plot the accumulated data
    colors = plt.colormaps.get_cmap("tab10")  # Get colors for the channels
    for idx, channel_choice in enumerate(channels_to_load):
        ax.plot(accumulated_df.index, accumulated_df[f'channel_{channel_choice}'], label=f'Channel {channel_choice}', linestyle='-', color=colors(idx))

    # Add vertical blue lines every final_sub_cycle_length and annotate
    num_sections = len(accumulated_df) // final_sub_cycle_length
    for j in range(num_sections + 1):
        if j * final_sub_cycle_length < len(accumulated_df):
            ax.axvline(x=accumulated_df.index[j * final_sub_cycle_length], color='blue', linestyle='--', alpha=0.7)

    # Annotate with anomaly percentages along the top x-axis
    #ylim = [0, 1]
    #ax.set_ylim(ylim)

    for j in range(num_sections):
        start_idx = j * final_sub_cycle_length
        end_idx = min(start_idx + final_sub_cycle_length, len(accumulated_df) - 1)
        mid_point = accumulated_df.index[start_idx + (end_idx - start_idx) // 2]
        anomaly_percentage = accumulated_df.iloc[start_idx:end_idx]['anomaly_percentage'].mean()
        #ax.text(mid_point, ylim[1] + (ylim[1] * 0.05), f'{anomaly_percentage:.2f}%', color='black', fontsize=12, verticalalignment='bottom', horizontalalignment='center')
        
        # Convert data coordinates to normalized axes coordinates for annotation
        x_coord = (mid_point - accumulated_df.index[0]) / (accumulated_df.index[-1] - accumulated_df.index[0])
        ax.annotate(f'{anomaly_percentage:.2f}%', xy=(x_coord, 1.05), xycoords='axes fraction', color='black', fontsize=12, verticalalignment='bottom', horizontalalignment='center')
    
       # Annotate with channel error ratios and percentages for the section
        for idx, channel_choice in enumerate(channels_to_load):
            channel_error_ratio = accumulated_df.iloc[start_idx:end_idx][f'error_ratio_{channel_choice}'].mean()
            #channel_error_percentage = accumulated_df.iloc[start_idx:end_idx][f'error_percentage_{channel_choice}'].mean()
            #channel_error_percentage = accumulated_df.iloc[start_idx:end_idx][f'error_{channel_choice}'].mean() * 100

            #ax.text(mid_point, ylim[1] + (ylim[1] * 0.1 + idx * 0.05), f'{channel_choice}: {channel_error_ratio:.2f}', color=colors(idx), fontsize=10, verticalalignment='bottom', horizontalalignment='center')
            y_coord = 1.10 + idx * 0.05
            ax.annotate(f'{channel_choice}: {channel_error_ratio:.2f}', xy=(x_coord, y_coord), xycoords='axes fraction', color=colors(idx), fontsize=10, verticalalignment='bottom', horizontalalignment='center')


    #ax.set_ylim(ylim)  # Reset y-axis limits to avoid adjusting them for text        
    # Handle legend labels (to avoid duplicate labels in the legend)
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)

    ax.set_title('Measurements Over Time with Anomalies for Multiple Channels')
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Measurement Value')
    ax.grid(True)
    plt.setp(ax.get_xticklabels(), rotation=45)
    fig.tight_layout()

    # Display the plot
    clear_output(wait=True)
    display(fig)

# Show final plot
plt.show()
